In [1]:
# Import libraries.
import logging
import pickle
import pandas as pd
import logging
import os

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

d:\Documents\PROJECTS\Python Programming\Data\mlops_dependencies\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Documents\PROJECTS\Python Programming\Data\mlops_dependencies\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [4]:
def ride_predictions(year, month):

    df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

    dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(dicts)
    y_pred = lr.predict(X_val)

    print(f'The Mean taxi ride duration predicted for {year:04d}/{month:02d} is: {y_pred.mean()}')

    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    df['predictions'] = y_pred

    df_result = df[['ride_id', 'predictions']].copy()

    # Write results to a parquet file.
    if not os.path.exists('./data'):
        os.makedirs('./data')

    output_file = f'data/{year:04d}-{month:02d}-predictions.parquet'

    df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
    )

In [5]:
ride_predictions(2021, 2)

The Mean taxi ride duration predicted for 2021/02 is: 16.191691679979066
